In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [3]:
prop = pickle.load(open("Data.p","rb"))

In [4]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72474 entries, 0 to 72473
Data columns (total 35 columns):
id                            72474 non-null object
created_on                    72474 non-null object
operation                     72474 non-null object
property_type                 72474 non-null object
place_name                    72446 non-null object
place_with_parent_names       72474 non-null object
country_name                  72474 non-null object
state_name                    72474 non-null object
geonames_id                   62030 non-null float64
lat-lon                       48174 non-null object
lat                           48174 non-null float64
lon                           48174 non-null float64
price                         72474 non-null float64
currency                      72473 non-null object
price_aprox_local_currency    72474 non-null float64
price_aprox_usd               72474 non-null float64
surface_total_in_m2           72474 non-null float64


In [5]:
#OJO CON ESTO
proper = pickle.load(open("porpiedadesPruebaConColTransGas.p", "rb"))

In [6]:
proper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 25 columns):
id                           14166 non-null int64
created_on                   14166 non-null object
property_type                14166 non-null object
operation                    14166 non-null object
place_name                   14166 non-null object
place_with_parent_names      14166 non-null object
country_name                 14166 non-null object
state_name                   14166 non-null object
lat-lon                      10487 non-null object
lat                          10487 non-null float64
lon                          10487 non-null float64
surface_total_in_m2          11853 non-null float64
surface_covered_in_m2        13005 non-null float64
floor                        1368 non-null float64
rooms                        7500 non-null float64
expenses                     2543 non-null object
description                  14166 non-null object
geonames_id           

In [7]:
#agrego el set de pruebas con los datos posta al de entrenamiento
propTOT = pd.merge(prop, proper, how="outer")
len(propTOT)

86640

In [8]:
#cortamos por precios
propTOT = propTOT[(propTOT['price_usd_per_m2']>150) & (propTOT['price_usd_per_m2']<8000)]
len(propTOT)

77775

In [9]:
propTOT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77775 entries, 0 to 81159
Data columns (total 35 columns):
id                            77775 non-null object
created_on                    77775 non-null object
operation                     77775 non-null object
property_type                 77775 non-null object
place_name                    77747 non-null object
place_with_parent_names       77775 non-null object
country_name                  77775 non-null object
state_name                    77775 non-null object
geonames_id                   66898 non-null float64
lat-lon                       52651 non-null object
lat                           52651 non-null float64
lon                           52651 non-null float64
price                         70655 non-null float64
currency                      70655 non-null object
price_aprox_local_currency    70655 non-null float64
price_aprox_usd               77775 non-null float64
surface_total_in_m2           77710 non-null float64


In [11]:
#completo las superficies totales con las cubiertas
propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[17] if(np.all(pd.isnull(row[16]))) else row[16], axis=1)

## Division del set de Entrenamiento

In [12]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Se le asigna un valor numerico al tipo de propiedad

In [13]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [14]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [15]:
def asignarPV(name, dic):
    return dic[name]   

In [16]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=True)

In [17]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*100
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [18]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [19]:
pEnt = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

In [20]:
len(pEnt)

77492

# Se carga el set de Prueba

In [21]:
proper = pickle.load(open("porpiedadesPruebaConColTransGas.p", "rb"))

In [22]:
len(proper)

14166

In [23]:
#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [24]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

In [25]:
HGdics

{' Country Maschwitz Club': 21514.06265777542,
 ' los alamos': 36076.56240115341,
 'Abasto': 41355.367451781036,
 'Abril Club de Campo': 7578.110849733132,
 'Acacias Blancas': 15234.180184029135,
 'Acassuso': 40758.475176322514,
 'Adrogu\xc3\xa9': 32188.41046750249,
 'Aeropuerto Internacional Ezeiza': 24405.19510129775,
 'Agronom\xc3\xada': 42349.0029699105,
 'Albanueva Barrio Cerrado': 45930.943975769995,
 'Aldo Bonzi': 12645.705688835964,
 'Alejandro Korn': 14138.278243489904,
 'Alejandro Peti\xc3\xb3n': 10857.57230488479,
 'Almagro': 43544.9145233115,
 'Almirante Brown': 28896.605612881718,
 'Altamira': 40060.77861714203,
 'Altos de Hudson II': 41058.15622453175,
 'Altos de Manzanares 1 y 2': 29994.721802870456,
 'Altos de Matheu': 15532.827611628498,
 'Altos del Golf': 14337.335103615955,
 'Altos del Pilar': 43644.20710272337,
 'Armenia Country Club': 26400.520352840344,
 'Avellaneda': 34083.820889225244,
 'Bah\xc3\xada del Sol': 45136.20621487863,
 'Balvanera': 40159.81041934995,


In [26]:
def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

In [27]:
#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

In [28]:
#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

In [30]:
pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value']]

# PRUEBA Random Forest Regresivo

In [31]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'lat','lon','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],pEnt[['price_aprox_usd']],test_size=0.0)

In [33]:
#definir el modelo a probar
model = RandomForestRegressor(n_estimators=1000, max_features='log2', min_samples_leaf=2, min_samples_split=4, bootstrap=True, oob_score=True)
n = 1000

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_aprox_usd'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [58]:
pPru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 8 columns):
property_type                14166 non-null int64
surface_total_in_m2          13392 non-null float64
lat                          12586 non-null float64
lon                          12586 non-null float64
Cant_ColeYUniv               12586 non-null float64
Cant_LocalesGastronomicos    12586 non-null float64
Cant_ParadasTransporte       12586 non-null float64
place_value                  14156 non-null float64
dtypes: float64(7), int64(1)
memory usage: 885.4 KB


In [34]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=0)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru)

In [39]:
pPru['price_usd'] = model.predict(pPru_imp)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
pPru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 9 columns):
property_type                14166 non-null int64
surface_total_in_m2          13392 non-null float64
lat                          12586 non-null float64
lon                          12586 non-null float64
Cant_ColeYUniv               12586 non-null float64
Cant_LocalesGastronomicos    12586 non-null float64
Cant_ParadasTransporte       12586 non-null float64
place_value                  14156 non-null float64
price_usd_per_m2             14166 non-null float64
dtypes: float64(8), int64(1)
memory usage: 996.1 KB


In [37]:
result = pd.DataFrame()

In [38]:
result["id"] = proper["id"]

In [40]:
result["price_usd"] = pPru['price_usd']

In [41]:
result.head()

,id,price_usd
0,2316505,100740.829984
1,2320880,175592.750979
2,2321742,216809.696688
3,2276796,94342.544866
4,2330364,77784.871841


In [42]:
result.to_csv(index=False, path_or_buf="results4PhiAlpha.csv")

In [40]:
#para ver mas o menos que tan lejos quedo
result["real_price_usd"] = proper['price_aprox_usd']

In [42]:
result.head()

,id,price_usd,real_price_usd
0,2316505,217547.459120,11111.00
1,2320880,194063.271378,11111.00
2,2321742,172941.927841,12554.74
3,2276796,81491.590068,13000.00
4,2330364,63278.760907,13000.00
